# Distributed Energy Resources Cost Optimization using Gurobi Python API

### Note 
The data used below is just for the purpose of implementing the Gurobi optimizer. This Gurobi optimizer solution is of low to medium difficulty. The intention of this Jupyter notebook is to demonstrate the use of Python and Gurobi API for implementing optimization solution.

## Objective

This work takes into account the consumers individual objectives considering multiple criteria. It is challenging for consumers to compare electricity purchases across difference metrics.

The model addresses optimal allocation and pricing of generation by DER. As this is a question of short-term (operational) dispatch rather than long-term planning, investment decisions and corresponding costs are not part of the modeling framework.

It selects an optimal set of energy sources to satisfy anticipated power demand over a 24-hour time horizon.

This model is of intermediate difficulty and implemented using Python and the Gurobi Python API.

---
## Problem Statement
Energy sources are grouped into three distinct types - grid, PV, battery, with different characteristics for each type (power output, cost per unit hour, startup cost, etc.). A source can be on or off, with a startup cost associated with transitioning from off to on, and power output that can fall anywhere between a specified minimum and maximum value when the unit is on. A 24-hour time horizon is divided into 5 discrete time periods, each with an expected total power demand. The model decides which units to turn on, and when, in order to satisfy demand for each time period. The model also captures a reserve requirement, where the selected sources must be capable of increasing their output, while still respecting their maximum output, in order to cope with the situation where actual demand exceeds predicted demand.

Anticipated demand is as follows:

| Time Period | Demand (megawatts) |
| --- | --- |
| 12 pm to 6 am | 15000 |
| 6 am to 9 am | 30000 |
| 9 am to 3 pm | 25000 |
| 3 pm to 6 pm | 40000 |
| 6 pm to 12 pm | 27000 |

Sources are grouped into three types PV, battery and grid, with the following minimum and maximum output for each type (when they are on):

| Type | Number available | Minimum output (MW) | Maximum output (MW) |
| --- | --- | --- | --- |
| 0 | 12 |  850 | 2000 |
| 1 | 10 | 1250 | 1750 |
| 2 | 5 | 1500 | 4000 |

There are costs associated with using a sources: a cost per hour when the source is on (and generating its minimum output), a cost per kW hour in dollars above its minimum:

| Type | Cost per hour (when on) | Cost per MWh above minimum | Startup cost |
| --- | --- | --- | --- |
| 0 | $\$1000$ | $\$2.00$ | $\$2000$ |
| 1 | $\$2600$ | $\$1.30$ | $\$1000$ |
| 2 | $\$3000$ | $\$3.00$ | $\$500$ |

Sources must meet predicted demand, but they must also have sufficient reserve capacity to be able to cope with the situation where actual demand exceeds predicted demand. For this model, the set of selected sources must be able to produce as much as 115% of predicted demand.

Which sources should be committed to meeting anticipated demand in order to minimize total cost?

---
## Model Formulation

### Sets and Indices

$t \in \text{Types}=\{0,1,2\}$: Types of generators.

$p \in \text{Periods}=\{0,1,2,3,4\}$: Time periods.

### Parameters

$\text{period_hours}_p \in \mathbb{N}^+$: Number of hours in each time period.

$\text{generators}_t \in \mathbb{N}^+$: Number of generators of type $t$.

$\text{demand}_p \in \mathbb{R}^+$: Total power demand for time period $p$.

$\text{start0} \in \mathbb{N}^+$: Number of generators that are on at the beginning of the time horizon (and available in time period 0 without paying a startup cost).

$\text{min_output}_t \in \mathbb{R}^+$: Minimum output for generator type $t$ (when on).

$\text{max_output}_t \in \mathbb{R}^+$: Maximum output for generator type $t$.

$\text{base_cost}_t \in \mathbb{R}^+$: Minimum operating cost (per hour) for a generator of type $t$.

$\text{per_mw_cost}_t \in \mathbb{R}^+$: Cost to generate one additional MW (per hour) for a generator of type $t$.

$\text{startup_cost}_t \in \mathbb{R}^+$: Startup cost for generator of type $t$.

### Decision Variables

$\text{ngen}_{t,p} \in \mathbb{N}^+$: Number of generators of type $t$ that are on in time period $p$.

$\text{output}_{t,p} \in \mathbb{R}^+$: Total power output from generators of type $t$ in time period $p$.

$\text{nstart}_{t,p} \in \mathbb{N}^+$: Number of generators of type $t$ to start in time period $p$.



### Objective Function

- **Cost**: Minimize the cost (in USD) to satisfy the predicted electricity demand.

\begin{equation}
\text{Minimize} \quad Z_{on} + Z_{extra} + Z_{startup}
\end{equation}

\begin{equation}
Z_{on} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{base_cost}_t*\text{ngen}_{t,p}}
\end{equation}
\begin{equation}
Z_{extra} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{per_mw_cost}_t*(\text{output}_{t,p} - \text{min_load}_t})
\end{equation}
\begin{equation}
Z_{startup} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{startup_cost}_t*\text{nstart}_{t,p}}
\end{equation}


### Constraints

- **Available generators**: Number of generators used must be less than or equal to the number available.

\begin{equation}
\text{ngen}_{t,p} \leq \text{generators}_{t} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **Demand**: Total power generated across all generator types must meet anticipated demand for each time period $p$.

\begin{equation}
\sum_{t \in \text{Types}}{\text{output}_{t,p}} \geq \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **Min/max generation**: Power generation must respect generator min/max values.

\begin{equation}
\text{output}_{t,p} \geq \text{min_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

\begin{equation}
\text{output}_{t,p} \leq \text{max_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **Reserve**: Selected generators must be able to satisfy demand that is as much as 15% above the prediction.

\begin{equation}
\sum_{t \in \text{Types}}{\text{max_output}_t*\text{ngen}_{t,p}} \geq 1.15 * \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **Startup**: Establish relationship between number of active generators and number of startups (use $start0$ for period before the time horizon starts)

\begin{equation}
\text{ngen}_{t,p} \leq \text{ngen}_{t,p-1} + \text{startup}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

## Input Data
We define all the input data of the model.

In [2]:
# Parameters

ntypes = 3
nperiods = 5
maxstart0 = 5

generators = [12, 10, 5]
period_hours = [6, 3, 6, 3, 6]
demand = [15000, 30000, 25000, 40000, 27000]
min_load = [850, 1250, 1500]
max_load = [2000, 1750, 4000]
base_cost = [1000, 2600, 3000]
per_mw_cost = [2, 1.3, 3]
startup_cost = [2000, 1000, 500]

## Model Deployment

We create a model and the variables. For each time period, we have: an integer decision variable to capture the number of active generators of each type (ngen), an integer variable to capture the number of generators of each type we must start (nstart), and a continuous decision variable to capture the total power output for each generator type (output).

In [3]:
model = gp.Model('PowerGeneration')
ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")

Restricted license - for non-production use only - expires 2022-01-13


Next we insert the constraints:

The number of active generators can't exceed the number of generators.

In [4]:
# Generator count

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))

Total power output for a generator type depends on the number of generators of that type that are active.

In [5]:
# Respect minimum and maximum output per generator type

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

Total output for each time period must meet predicted demand.

In [6]:
# Meet demand

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) >= demand[period]
                               for period in range(nperiods))

Selected generators must be able to cope with an excess of demand.

In [7]:
# Provide sufficient reserve capacity

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >= 1.15*demand[period]
                    for period in range(nperiods))

Connect the decision variables that capture active generators with the decision variables that count startups.

In [8]:
# Startup constraint

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))

Objective: minimize total cost.  Cost consists of three components: the cost for running active generation units, the cost to generate power beyond the minimum for each unit, and the cost to start up generation units.

In [9]:
# Objective: minimize total cost

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj)

Next, we start the optimization and Gurobi finds the optimal solution.

In [10]:
model.write('junk.lp')
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 70 rows, 45 columns and 147 nonzeros
Model fingerprint: 0x08cf02b0
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+04]
Presolve removed 16 rows and 1 columns
Presolve time: 0.00s
Presolved: 54 rows, 44 columns, 130 nonzeros
Variable types: 0 continuous, 44 integer (0 binary)
Found heuristic solution: objective 1123913.3000
Found heuristic solution: objective 1084300.0000

Root relaxation: objective 9.995143e+05, 23 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 999514.286    0    7 1084300.00 999514.286  7.82%     -    0s
H    0     0   

---
## Analysis

The anticipated demand for electricity over the 24-hour time window is shown above. The detailed plan for each time period is as follows.

### Unit Commitments

The following table shows the number of generators of each type that are active in each time period in the optimal solution:

In [11]:
rows = ["Type" + str(t) for t in range(ntypes)]
units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        units.loc["Type"+str(t), p] = ngen[t,p].x
units

,0,1,2,3,4
Type0,12.0,12.0,12.0,12.0,12.0
Type1,3.0,8.0,8.0,9.0,9.0
Type2,-0.0,-0.0,-0.0,2.0,-0.0


The following shows the number of generators of each type that must be started in each time period to achieve this plan:

In [12]:
startups = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        startups.loc["Type"+str(t), p] = int(nstart[t,p].x)
startups

,0,1,2,3,4
Type0,7.0,0.0,0.0,0.0,0.0
Type1,0.0,5.0,0.0,1.0,0.0
Type2,0.0,0.0,0.0,2.0,0.0


### Conclusion

Thus, using Gurobi Python API and Python, we can implement the cost optimization. 

As described in the word document shared in the email, we can formulate various models, write their constraints, sets, define input data and use the Gurobi optimizer. 

Gurobi Optimizer can be used for following major new features; 


- Non-Convex Quadratic Optimization: With the addition of a new bilinear solver, the Gurobi Optimizer now supports non-convex quadratic optimization. This groundbreaking new capability allows users to solve problems with non-convex quadratic constraints and objectives – enabling them to find globally optimal solutions to classic bilinear pooling and blending problems.

- Piecewise-Linear (PWL) Constraints: Users can now add PWL constraints directly as a modeling construct instead of having to represent them as a combination of linear and SOS constraints – thereby greatly simplifying their use.

- Function Constraints with Automatic PWL Translation: With Gurobi 9.0, users can add nine basic, nonlinear, univariate function constraints – including polynomial, exponential, logarithmic, and trigonometric functions – and the Gurobi Optimizer will perform an automatic piecewise-linear approximation of these functions.

- Python Matrix API: The Gurobi Optimizer’s Python interface has been extended to support matrix-oriented modeling using NumPy or SciPy matrices.

- MIP Scenario Analysis: This new feature allows users to construct multiple scenarios and run them simultaneously.

- New Solution Improvement Heuristic: Gurobi 9.0 provides a new, more effective heuristic for focusing on improving MIP solutions.

- Intermediate Solution Files: Users are now able to write solutions found during the MIP solution process to files as soon as they are discovered.